In [1]:
import numpy as np
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('indicators_db.sqlite3')

In [4]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM sqlite_master', conn)

In [5]:
indicators

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...
2,view,v_org_indicators_all,v_org_indicators_all,0,CREATE VIEW v_org_indicators_all AS\r\nselect ...


In [6]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)

In [7]:
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


## Задание 1
Для каждой организации из main.csv  
подтяните выручку и число сотрудников за 2020, 2021 и 2022.  
Если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их.  
Посчитайте дельту по выручке и сотрудникам между 2022 и 2020.  
Отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID.  
Названия колонок сделайте на русском языке.

In [3]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


In [4]:
indicators = indicators[(indicators.INDICATOR_NAME == 'Выручка') | (indicators.INDICATOR_NAME == 'Число сотрудников')]

In [5]:
indicators = indicators.pivot_table('VALUE', ['ID'], ['INDICATOR_NAME', 'YEAR'])

In [6]:
indicators.columns = [col[0] + ' ' + str(col[1]) for col  in indicators.columns.values]

In [7]:
indicators.head()

,Выручка 2020,Выручка 2021,Выручка 2022,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022
ID,,,,,,
19630,31222228.0,21721921.0,NaN,NaN,15.0,9.0
19643,35670701.0,39125612.0,NaN,17.0,NaN,NaN
19647,34926614.0,23176004.0,27925131.0,16.0,15.0,NaN
19657,23410468.0,28065717.0,36074059.0,NaN,8.0,12.0
19677,NaN,30803884.0,25303592.0,17.0,9.0,NaN


In [8]:
indicators = indicators.dropna(subset=['Выручка 2020', 'Выручка 2022', 'Число сотрудников 2020', 'Число сотрудников 2022'])

In [9]:
indicators.loc[:, 'Дельта по выручке'] = indicators['Выручка 2022'] - indicators['Выручка 2020']
indicators.loc[:, 'Дельта по сотрудникам'] = indicators['Число сотрудников 2022'] - indicators['Число сотрудников 2020']

In [10]:
indicators = indicators.reset_index()

In [11]:
df_main = pd.read_csv('6_main.csv')
df_main

,ID,Name,Address
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15"
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ..."
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро..."
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро..."
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро..."
...,...,...,...
9995,192373,Рамен Клаб,"город Москва, Мясницкая улица, дом 16"
9996,204304,Папа Джонс,"город Москва, проспект Маршала Жукова, дом 59"
9997,301366,Up Sushi,"город Москва, город Московский, улица Москвити..."
9998,339278,Шаурма,"город Москва, улица Орджоникидзе, дом 12, стро..."


In [12]:
indicators = indicators.merge(df_main)

In [13]:
indicators = indicators.iloc[:, [0, -2, -1, 1, 2, 3, -4, 4, 5, 6, -3]]

In [14]:
indicators = indicators.rename(columns={'ID': 'Идентификатор', 'Name': 'Наименование', 'Address': 'Адрес'})

In [15]:
indicators = indicators.sort_values(['Дельта по выручке', 'Идентификатор'], ascending=[False, True])

In [17]:
indicators.to_csv('./6_5_1.csv', index=False)

## Задание 2
Коллеги уже пишут вам в чат и просят посчитать суммарную выручку по организациям за 2022 год: 357706, 198181, 27549.  
Через несколько минут приходит еще одно сообщение, просят добавить к ним еще четыре: 217895, 201139, 357706, 20105.  
Вы почти успели импортировать pandas и коллеги снова просят добавить к ним еще организаций из списка.  
Посчитайте, пожалуйста, суммарную выручку за 2022 год. Если у каких-то организаций нет выручки за 2022 год, то возьмите самую свежую доступную (за другой год). Ответ округлите до целого

In [30]:
df = pd.read_excel('6_Выручка_по_компаниям.xlsx', header=0, sheet_name='Лист1')

In [40]:
df = pd.concat([
    df, 
    pd.DataFrame([357706, 198181, 27549], columns=['Айдии']), 
    pd.DataFrame([217895, 201139, 357706, 20105], columns=['Айдии'])
])\
.drop_duplicates()\
.reset_index(drop=True)